This notebook checks the behaviour of the embedding space

In [1]:
import re
import os
import sys
import pickle
import random
import glob
import scipy.sparse

import numpy as np
import pandas as pd

from annoy import AnnoyIndex
from scipy.spatial import distance

scriptpath = "../../"
sys.path.append(os.path.abspath(scriptpath))
from helpers.helpers import *

In [2]:
COMMON_PATH = '/dlabdata1/youtube_large/jouven/word2vecf_preprocessing'

In [3]:
'''
Retrieve the array obtained by apllying the dimentionality reduction algorithm
graph_matrix: SHAPE: (channels, n_comp)

PARAMETER:
    - file_path: the path where the embedding graph is stored

RETURN: 
    - df: DataFrame representing the graph in the embedding space
'''
def get_dataframe_in_embedding_space(filename):
    graph_matrix = np.load(os.path.join(COMMON_PATH, filename + '.npy'))
    df = pd.DataFrame(graph_matrix)
    df = df.rename(lambda x: 'dr'+str(x), axis='columns')
    df['index'] = np.load(os.path.join(COMMON_PATH, filename + 'index_ordering.npy')).astype(int)
    df = df.sort_values(by=['index'])
    return df.set_index('index')


In [4]:
# Selected channels and id-index mapping
dict_channel_ind, dict_ind_channel, channels_id = filtered_channels_index_id_mapping()

### Find k closest channels using annoy library

First to check how good is the embedding space, we are going to choose a channel and it's k closest channels in the embedding space. By looking at these channels in the YouTube website, we should get similar channels if the embedding space is good.

In [14]:
channelcrawler = pd.read_csv("/dlabdata1/youtube_large/channelcrawler.csv")
channelcrawler['channel_id'] = channelcrawler['link'].str.split('/').str[-1]
channelcrawler.head()

,category,join_date,link,name,subscribers,videos,channel_id
0,Film and Animation,2017-05-21,http://www.youtube.com/channel/UCBJuEqXfXTdcPS...,MagnusNation,65100,28,UCBJuEqXfXTdcPSbGO9qqn1g
1,Entertainment,2011-12-13,http://www.youtube.com/channel/UCkNW9Q1VR_aeZ6...,Mago Dario Animazion...,60200,48,UCkNW9Q1VR_aeZ6uht83jJVQ
2,Music,2013-09-13,http://www.youtube.com/channel/UC1xcnrpcF59FWW...,Mägo de Oz - Topic,40200,395,UC1xcnrpcF59FWWELtZvJTdg
3,Music,2008-03-17,http://www.youtube.com/channel/UCXhkGgooXHDNwg...,Mago Merlino,14800,838,UCXhkGgooXHDNwgJXmoTSN7g
4,Entertainment,2014-10-19,http://www.youtube.com/channel/UCvZGsuvKlYOGiZ...,MAGO TOMÁS,26200,31,UCvZGsuvKlYOGiZTsxwJNS5Q


In [15]:
channelcrawler = channelcrawler[channelcrawler['channel_id'].apply(lambda row: row in channels_id)]

In [16]:
channelcrawler['category'].unique()

array(['Comedy', 'Gaming', 'Film and Animation', 'Science & Technology',
       'Music', 'Education', 'Sports', 'People & Blogs', 'Entertainment',
       'Nonprofits & Activism', 'News & Politics', 'Travel & Events',
       'Howto & Style', 'Autos & Vehicles', 'Pets & Animals', nan],
      dtype=object)

In [17]:
channelcrawler[channelcrawler['category'] == 'Film and Animation'][50:100]

,category,join_date,link,name,subscribers,videos,channel_id
3767,Film and Animation,2015-11-13,http://www.youtube.com/channel/UCWkfvm1YaPY8H1...,IAmZamber Reacts,44900,510,UCWkfvm1YaPY8H1rQY6fB60Q
3787,Film and Animation,2015-03-08,http://www.youtube.com/channel/UCBQhLER2rVxN2T...,MSK Movies,1080000,4166,UCBQhLER2rVxN2TkSuPfJ6rA
3791,Film and Animation,2016-07-08,http://www.youtube.com/channel/UC_CstbFmNkvWAH...,Aluminium Plays,45900,50,UC_CstbFmNkvWAH3RLdxAusw
3799,Film and Animation,2012-02-12,http://www.youtube.com/channel/UCpW-R7T0kA4jFa...,Aluriya,39400,46,UCpW-R7T0kA4jFaB-GAQurXQ
3865,Film and Animation,2014-03-07,http://www.youtube.com/channel/UCSZLzCcpZqWJUx...,V KINGS,114000,300,UCSZLzCcpZqWJUxUi8h57sew
3954,Film and Animation,2017-03-18,http://www.youtube.com/channel/UCmlU7184B6TBOy...,Bonda,173000,73,UCmlU7184B6TBOyJ2OhseCAg
3976,Film and Animation,2012-12-30,http://www.youtube.com/channel/UCDzMRfmdypVUxZ...,PushedToInsanity,531000,629,UCDzMRfmdypVUxZ0H8xvGDOA
3996,Film and Animation,2017-07-25,http://www.youtube.com/channel/UCLiVYcPG3bsVAw...,ohitscosmo,81500,49,UCLiVYcPG3bsVAwqhfTsr5kA
4015,Film and Animation,2010-04-13,http://www.youtube.com/channel/UCbsfyGlrjrKQC0...,Jared Owen,339000,47,UCbsfyGlrjrKQC0gbzK0-EiA
4074,Film and Animation,2015-07-14,http://www.youtube.com/channel/UC6DKFSj5aDJeMz...,Nebula Effect,119000,83,UC6DKFSj5aDJeMz0kmbbpu3w


In [5]:
'''
    Retrieve the array obtained by apllying the dimentinality reductin algorithm
    graph_matrix: SHAPE: (channels, n_comp)
    
    PARAMETERS:
        - df_embedding: DataFrame representing the graph in the embedding space
        
    RETURN: The annoy index
    '''
def get_annoy_index(df):
    index = AnnoyIndex(df.shape[1], "euclidean")  # Length of item vector that will be indexed
    df.apply(lambda row: index.add_item(row.name, np.array(row)), axis = 1)
    index.build(100) # 100 trees
    return index

In [19]:
def get_k_nearest_neighbors(filename, ref_index_channel, k = 20):
    df = get_dataframe_in_embedding_space(filename)
    index = get_annoy_index(df)
    nearest_neighbors_index = index.get_nns_by_item(ref_index_channel, k,)
    nearest_neighbors_id = [dict_ind_channel[val] for val in nearest_neighbors_index]
    return nearest_neighbors_id

In [10]:
dict_channel_ind['UCLFW3EKD2My9swWH4eTLaYw']

19769

In [20]:
filename = 'channel_embedding_30M_dc_sample_0'

In [21]:
nearest_neighbors_id = get_k_nearest_neighbors(filename, dict_channel_ind['UCMpOz2KEfkSdd5JeIJh_fxw'], k= 40)
nearest_neighbors = pd.DataFrame(nearest_neighbors_id, columns = ['channel_id']).merge(channelcrawler)
nearest_neighbors

,channel_id,category,join_date,link,name,subscribers,videos
0,UCMpOz2KEfkSdd5JeIJh_fxw,Howto & Style,2009-01-18,http://www.youtube.com/channel/UCMpOz2KEfkSdd5...,Shaaanxo,3215340,1422
1,UC21yq4sq8uxTcfgIxxyE9VQ,Howto & Style,2011-06-27,http://www.youtube.com/channel/UC21yq4sq8uxTcf...,Carli Bybel,6190000,546
2,UCz0Qnv6KczUe3NH1wnpmqhA,Howto & Style,2006-11-27,http://www.youtube.com/channel/UCz0Qnv6KczUe3N...,Nicole Guerriero,2870000,664
3,UCDHQbU57NZilrhbuZNbQcRA,Howto & Style,2013-02-27,http://www.youtube.com/channel/UCDHQbU57NZilrh...,PatrickStarrr,4430000,358
4,UCxj0QizmFhx7kVKSArHBCTA,Howto & Style,2011-03-09,http://www.youtube.com/channel/UCxj0QizmFhx7kV...,Casey Holmes,1770000,730
5,UCERTKUMXl6x8G79BmG6KtpQ,Howto & Style,2010-06-17,http://www.youtube.com/channel/UCERTKUMXl6x8G7...,Meghan Rienks,2190000,761
6,UCJttQwY6KBhwov_2wKywtqA,Howto & Style,2010-06-27,http://www.youtube.com/channel/UCJttQwY6KBhwov...,NikkiPhillippi,1360000,959
7,UCf5Z8I0Yy0_-a-xAu2_0Yiw,Entertainment,2014-08-04,http://www.youtube.com/channel/UCf5Z8I0Yy0_-a-...,Sarah Baska,1580000,198
8,UCCgDVqiPU10shxzmwkMwJ6A,Howto & Style,2009-09-22,http://www.youtube.com/channel/UCCgDVqiPU10shx...,PiinkSparkles,1170000,1557
9,UCK2ACorzpH-igxuHZ2ObCEA,People & Blogs,2010-03-10,http://www.youtube.com/channel/UCK2ACorzpH-igx...,itsJudysLife,1660000,2893


In [22]:
nearest_neighbors_id = get_k_nearest_neighbors(filename, dict_channel_ind['UCAql2DyGU2un1Ei2nMYsqOA'], k= 300)
nearest_neighbors = pd.DataFrame(nearest_neighbors_id, columns = ['channel_id']).merge(channelcrawler)

In [23]:
nearest_neighbors[:50]

,channel_id,category,join_date,link,name,subscribers,videos
0,UCAql2DyGU2un1Ei2nMYsqOA,News & Politics,2015-03-17,http://www.youtube.com/channel/UCAql2DyGU2un1E...,Donald J Trump,189000,222
1,UC08wrceRCWhajGG4udhETww,Entertainment,2008-04-22,http://www.youtube.com/channel/UC08wrceRCWhajG...,Investigation Discov...,328843,1326
2,UCPsSnSR7ymSE8on3wafNbqg,News & Politics,2011-05-06,http://www.youtube.com/channel/UCPsSnSR7ymSE8o...,WSB-TV,81900,8316
3,UCry2gCVdntv6r661sKUiKXw,News & Politics,2014-05-16,http://www.youtube.com/channel/UCry2gCVdntv6r6...,WCCO - CBS Minnesota...,56100,28788
4,UC3dZNn_1-kP76SmIqy4rHSw,News & Politics,2006-04-05,http://www.youtube.com/channel/UC3dZNn_1-kP76S...,ChangeDaChannel,130000,662
5,UCSiTS5Fg9Cl6T5t_EPDFLFw,Sports,2015-12-04,http://www.youtube.com/channel/UCSiTS5Fg9Cl6T5...,Rated Red,388000,2545
6,UC13mSI38YWz5zfvxXDPpePA,News & Politics,2013-08-19,http://www.youtube.com/channel/UC13mSI38YWz5zf...,FOX 13 News - Tampa ...,89800,2471
7,UCWF0PiUvUi3Jma2oFgaiX2w,Entertainment,2015-11-10,http://www.youtube.com/channel/UCWF0PiUvUi3Jma...,VICELAND,1480000,1474
8,UCQpltQMhYFvyeS5M6P0Zg-Q,Comedy,2011-01-13,http://www.youtube.com/channel/UCQpltQMhYFvyeS...,Creationist Cat,77700,246
9,UC6B_HuuSaR-IDsKuJVK0LEw,Entertainment,2016-03-16,http://www.youtube.com/channel/UC6B_HuuSaR-IDs...,DL Hughley,97400,548


In [76]:
path = '/dlabdata1/youtube_large/jouven/word2vecf_preprocessing/channel_embedding.npy'
nearest_neighbors_id = get_k_nearest_neighbors(path, dict_channel_ind['UC9pXxdNqCc2zjgRXSoowNNg'], k= 20)
nearest_neighbors = pd.DataFrame(nearest_neighbors_id, columns = ['channel_id']).merge(channelcrawler)
nearest_neighbors

57004


,channel_id,category,join_date,link,name,subscribers,videos
0,UC9pXxdNqCc2zjgRXSoowNNg,News & Politics,2014-07-09,http://www.youtube.com/channel/UC9pXxdNqCc2zjg...,A Blue Dot In Texas,10600,646
1,UCqSR91bTcQyrSZpUdGpJy1w,News & Politics,2008-12-31,http://www.youtube.com/channel/UCqSR91bTcQyrSZ...,Julia Blog,10300,376
2,UCRPdcY4_Al8TrRtgkMzkrQQ,Entertainment,2014-03-21,http://www.youtube.com/channel/UCRPdcY4_Al8TrR...,Tony Dortie - 24/7 E...,98700,333
3,UCXEjC-1iUa1WhwrBp0eBBuw,Entertainment,2016-10-13,http://www.youtube.com/channel/UCXEjC-1iUa1Whw...,Be Less Stupid,52100,309
4,UCuDv5p8E-evaRSh542hDV5g,News & Politics,2015-06-16,http://www.youtube.com/channel/UCuDv5p8E-evaRS...,Robert Reich,149000,221
5,UCWCtoOXSwTcbVjOiK8iFGUg,People & Blogs,2015-06-10,http://www.youtube.com/channel/UCWCtoOXSwTcbVj...,Washington Watch,25000,164
6,UCwlHzGKZA-e2GeLMjIE51uw,News & Politics,2017-08-16,http://www.youtube.com/channel/UCwlHzGKZA-e2Ge...,Parody Project,50400,99
7,UCx_SjDi4CS5ALkWCS9ffldQ,News & Politics,2012-01-24,http://www.youtube.com/channel/UCx_SjDi4CS5ALk...,WGBH News,17700,5938
8,UCcRDioQprYoceT_3NhrTIAQ,Science & Technology,2013-11-11,http://www.youtube.com/channel/UCcRDioQprYoceT...,Brad Reed,13600,935
9,UC1Z1i5ClfrJhyu9z1hlmDkQ,News & Politics,2015-05-14,http://www.youtube.com/channel/UC1Z1i5ClfrJhyu...,Daily Kos,10800,878


In [24]:
nearest_neighbors_id = get_k_nearest_neighbors(filename, dict_channel_ind['UCAuUUnT6oDeKwE6v1NGQxug'], k= 20)
nearest_neighbors = pd.DataFrame(nearest_neighbors_id, columns = ['channel_id']).merge(channelcrawler)
nearest_neighbors

,channel_id,category,join_date,link,name,subscribers,videos
0,UCAuUUnT6oDeKwE6v1NGQxug,People & Blogs,2006-12-06,http://www.youtube.com/channel/UCAuUUnT6oDeKwE...,TED,14800000,3105
1,UCvB3solmhqtgDeLpD-yTtfg,Sports,2007-01-06,http://www.youtube.com/channel/UCvB3solmhqtgDe...,hickok45,4370000,1809
2,UCWJHDMgKWWvOsdyRF3HPVEw,Science & Technology,2008-02-26,http://www.youtube.com/channel/UCWJHDMgKWWvOsd...,Iraqveteran8888,2210000,1452
3,UCfAOh2t5DpxVrgS9NQKjC7A,Comedy,2006-03-14,http://www.youtube.com/channel/UCfAOh2t5DpxVrg...,The Onion,1360000,1390
4,UCfGmaA-nXPryTfimsnkLieQ,Entertainment,2013-12-07,http://www.youtube.com/channel/UCfGmaA-nXPryTf...,Chibi Reviews,373000,7048
5,UCvQECJukTDE2i6aCoMnS-Vg,Education,2006-10-01,http://www.youtube.com/channel/UCvQECJukTDE2i6...,Big Think,2650000,11135
6,UCsT0YIqwnpJCM-mx7-gSA4Q,Nonprofits & Activism,2009-06-23,http://www.youtube.com/channel/UCsT0YIqwnpJCM-...,TEDx Talks,20700000,141627
7,UCrTNhL_yO3tPTdQ5XgmmWjA,Entertainment,2007-09-29,http://www.youtube.com/channel/UCrTNhL_yO3tPTd...,RedLetterMedia,1050000,582
8,UC0aVoboXBUx2-tVIWHc3W2Q,Film and Animation,2009-04-24,http://www.youtube.com/channel/UC0aVoboXBUx2-t...,MILO,858000,803
9,UC1CSCMwaDubQ4rcYCpX40Eg,Gaming,2010-03-03,http://www.youtube.com/channel/UC1CSCMwaDubQ4r...,Gopher,476337,3843


In [25]:
nearest_neighbors_id = get_k_nearest_neighbors(filename, dict_channel_ind['UCwTkM6CvIsYFaFiMKIKCqHw'], k= 20)
nearest_neighbors = pd.DataFrame(nearest_neighbors_id, columns = ['channel_id']).merge(channelcrawler)
nearest_neighbors

,channel_id,category,join_date,link,name,subscribers,videos
0,UCwTkM6CvIsYFaFiMKIKCqHw,Film and Animation,2005-11-27,http://www.youtube.com/channel/UCwTkM6CvIsYFaF...,James Bond 007,115000,407
1,UCni4MJAf7XfS159w3_vWlRw,Entertainment,2016-09-01,http://www.youtube.com/channel/UCni4MJAf7XfS15...,The Order of the Gre...,55200,164
2,UCbdDw5mbJ6MvRt9Kj7mH2Vw,Gaming,2010-12-11,http://www.youtube.com/channel/UCbdDw5mbJ6MvRt...,BloodBurgerOne,16700,1550
3,UCcwQiDfUgWCMaHYMx7W5m3Q,Gaming,2006-08-11,http://www.youtube.com/channel/UCcwQiDfUgWCMaH...,Oppaiman100,17400,5091
4,UCy1Rr-GkiL8vlPApKsw6SuA,Sports,2011-08-03,http://www.youtube.com/channel/UCy1Rr-GkiL8vlP...,Central Coast Disc G...,80400,1143
5,UC_0kXiCbAWNPGkR7ndVbsfA,Film and Animation,2006-10-21,http://www.youtube.com/channel/UC_0kXiCbAWNPGk...,TheSpringwoodSlasher...,25200,280
6,UC13043Ga8_N3kItV22oHazQ,Gaming,2015-04-03,http://www.youtube.com/channel/UC13043Ga8_N3kI...,distant Observer,51900,151
7,UCIOcPxMwytfvKeq8M9lj80w,Music,2011-05-14,http://www.youtube.com/channel/UCIOcPxMwytfvKe...,Nate C. Concerts,17800,5559
8,UCkw6l3jIjVtrXom_xqftSMA,Gaming,2015-05-15,http://www.youtube.com/channel/UCkw6l3jIjVtrXo...,Prolike Chro,30800,623
9,UCgjvEc9WvlRRkMV0kXNyaig,Music,2007-11-23,http://www.youtube.com/channel/UCgjvEc9WvlRRkM...,HUMAN BEATBOX,64800,509


In [26]:
nearest_neighbors_id = get_k_nearest_neighbors(filename, dict_channel_ind['UCISF5OGuAtSLNF24TKTnXag'], k= 20)
nearest_neighbors = pd.DataFrame(nearest_neighbors_id, columns = ['channel_id']).merge(channelcrawler)
nearest_neighbors

,channel_id,category,join_date,link,name,subscribers,videos
0,UCISF5OGuAtSLNF24TKTnXag,Film and Animation,2013-12-17,http://www.youtube.com/channel/UCISF5OGuAtSLNF...,Magpiepony,736000,304
1,UCvcvDJOYYsZOwHi2UInJbqg,Music,2016-07-06,http://www.youtube.com/channel/UCvcvDJOYYsZOwH...,Undertale Peasant,193000,84
2,UCDhka7XkllGxop0_abgGfdA,Music,2010-07-02,http://www.youtube.com/channel/UCDhka7XkllGxop...,[CK9C] ChaoticCanine...,785000,158
3,UCvbepLqcdD8jqAv8P_NL2zA,People & Blogs,2014-10-10,http://www.youtube.com/channel/UCvbepLqcdD8jqA...,Hailey -Senpai,158000,250
4,UCARVbKs7x4zEm2vTDBiHatg,Entertainment,2014-12-14,http://www.youtube.com/channel/UCARVbKs7x4zEm2...,piaslittlecustoms,131000,770
5,UCIu4_D9HnABj9wZfHmlYlMg,Music,2015-08-03,http://www.youtube.com/channel/UCIu4_D9HnABj9w...,YouTubers Music,198134,187
6,UC7EFWpvc1wYuUwrtZ_BLi9A,Education,2015-01-26,http://www.youtube.com/channel/UC7EFWpvc1wYuUw...,Kids Learning Tube,539000,278
7,UCZkm79hXarnstAbkundHdmQ,Gaming,2015-11-28,http://www.youtube.com/channel/UCZkm79hXarnstA...,GellyJones,111000,226
8,UCBjh-uGy5RcclATnFpQBobw,Pets & Animals,2016-02-20,http://www.youtube.com/channel/UCBjh-uGy5RcclA...,A Chick Called Alber...,1567427,40
9,UCWXB965c8P0KzY9MmvvyEUQ,Gaming,2015-04-10,http://www.youtube.com/channel/UCWXB965c8P0KzY...,Captain Tate,681000,864


### Channels selected over the comments dataset
We randomly choose 10 000 users over the dataset.
For each user, we then pick two channels at random in the set of channels this user commented in.

In [31]:
'''
# Load the channel tuple sparse matrix
S = scipy.sparse.load_npz('/dlabdata1/youtube_large/jouven/final_sparse_matrix/sparse_matrix_word2vec_users_commented_geq_2_channels.npz')

# Sample NB_SAMPLE from the set of users
NB_SAMPLE = 3000
selected_users = random.sample(range(S.shape[1]), k = NB_SAMPLE)
S = S[:, selected_users]

# Create and store channels tuples
channels_tuple = []
for i in range(S.shape[1]):
    idx = S[:, i].nonzero()
    idx = list(idx[0])
    
    selected_channels = random.sample(idx, k = 2)
    channels_tuple.append((selected_channels[0], selected_channels[1]))
    
with open("/dlabdata1/youtube_large/jouven/channels_more_10k/channels_tuple_user_walk.pkl",'wb') as f:
     pickle.dump(channels_tuple, f)
f.close()
'''

In [33]:
'''
random_values_1 = random.sample(range(S.shape[0]), k = NB_SAMPLE)
random_values_2 = random.sample(range(S.shape[0]), k = NB_SAMPLE)
random_walk_channels = list(zip(random_values_1, random_values_2))

for ref, second in random_walk_channels:
    if ref == second:
        print('equal pair')
        break

with open("/dlabdata1/youtube_large/jouven/channels_more_10k/channels_tuple_random_walk.pkl",'wb') as f:
     pickle.dump(random_walk_channels, f)
f.close()
'''

#### Random walk

In [36]:
def get_random_walk(df_embedding):
    with open("/dlabdata1/youtube_large/jouven/channels_more_10k/channels_tuple_random_walk.pkl",'rb') as f:
         random_walk_channels = pickle.load(f)
    f.close()
    random_walk_distance = 0
    for val in random_walk_channels:
        random_walk_distance += distance.euclidean(df_embedding.iloc[val[0]], df_embedding.iloc[val[1]])
    return random_walk_distance

#### Compute metrics: users walk distance and relative nearest neighbor ranking 

In this section we want to measure the euclidian distance of a user walk compared to a random walk.

user walk: Euclidean distance in the embedding space between two randomly channels of a user.
random walk: Euclidean distance in the embedding space between two randomly channels.
position: Position of a channel taken from user u relatively of another channel taken from the same user in terms of its nearest neighbor ranking.


In [37]:
'''
Get the position of ref_channel relative to second_channel in terms of its nearest neighbors ranking.
PARAMETER:
    - ref_channel: The reference channel on which wwe compute it's k nearest neighbor
    - second_channel: The channel where we compute it's ranking relatively to ref_channel
    - dist: Euclidean distance between ref_channel and second_channel
    - index: annoy index
    - df_embedding: DataFrame representing the embedding space

RETURN: The position of second_channel relatively to ref_channel in terms of it's ranking

'''
def get_ranking_position_between_channels(ref_channel, second_channel, dist, index, df_embedding):
    
    nearest_neighbors_index = index.get_nns_by_item(ref_channel, len(df_embedding), search_k = 100000000)
    dist_k_th_nearest = distance.euclidean(df_embedding.iloc[ref_channel], 
                                           df_embedding.iloc[nearest_neighbors_index[len(nearest_neighbors_index)-1]])
    for i in range(0, len(nearest_neighbors_index)):
        if nearest_neighbors_index[i] == second_channel:
            return i
    

In [38]:
users_walk_tab = []
ranking_position_tab = []

len_random_set = len(channels_tuple)
len_embedding = len(channels_id)

files = ['channel_embedding_30M_dc_sample_0']
for file in files: 
    print('file ', file)
    df_embedding = get_dataframe_in_embedding_space(file)
    n_comp = df_embedding.shape[1]
    print('n_comp ', n_comp)
    random_walk_distance = get_random_walk(df_embedding)
        
    index = get_annoy_index(df_embedding)

    users_walk = 0
    ranking_position = 0

    for ref_channel, second_channel in channels_tuple:
        dist = distance.euclidean(df_embedding.iloc[ref_channel], df_embedding.iloc[second_channel])
        users_walk += dist
        ranking_position += get_ranking_position_between_channels(ref_channel, second_channel, dist, index, df_embedding)
    
    users_walk_tab.append(users_walk/random_walk_distance)
    ranking_position_tab.append(ranking_position/(len_random_set*len_embedding))


file  channel_embedding_30M_dc_sample_0
n_comp  200


In [39]:
users_walk_tab

[3.4931431686640346]

In [40]:
ranking_position_tab

[0.5126707190138704]

In [ ]:
'''
Plot the results obtained when computing the metrics
'''
import plotly.graph_objects as go

components_name = ['Features dim 50', 'Features dim 100', 'Features dim 200', 'Features dim 500']
colors = ['b', 'g,', 'r', 'c']

fig = go.Figure()

for i in range(len(components_name)):
    fig.add_trace(go.Scatter(
        x=ranking_position_tab,
        y=users_walk_tab,
        marker=dict(color=colors[i], size=12),
        mode="markers",
        name=components_name[i],
    ))

fig.update_layout(title="Gender Earnings Disparity",
                  xaxis_title="Ranking position",
                  yaxis_title="Users walk distance")

fig.show()